<a href="https://colab.research.google.com/github/rajaswa/Summer-Internship-IIT-Bombay-2019/blob/master/lda_reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dowloading Data

In [1]:
!wget --header="Host: codeload.github.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9,lb;q=0.8" --header="Referer: https://github.com/linanqiu/reddit-dataset" --header="Cookie: _octo=GH1.1.1541950956.1513609300; _ga=GA1.2.2088113052.1513609300; logged_in=yes; dotcom_user=rajaswa; _gat=1" --header="Connection: keep-alive" "https://codeload.github.com/linanqiu/reddit-dataset/zip/master" -O "CurlWget1684" -c

--2019-05-17 13:06:25--  https://codeload.github.com/linanqiu/reddit-dataset/zip/master
Resolving codeload.github.com (codeload.github.com)... 192.30.253.120
Connecting to codeload.github.com (codeload.github.com)|192.30.253.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘CurlWget1684’

CurlWget1684            [                 <=>] 242.66M  6.30MB/s    in 41s     

2019-05-17 13:07:06 (5.98 MB/s) - ‘CurlWget1684’ saved [254445121]



In [2]:
!unzip CurlWget1684

Archive:  CurlWget1684
d2f7dc81c1bae4238805ed5b20fdaaee45f71cde
   creating: reddit-dataset-master/
 extracting: reddit-dataset-master/.gitignore  
  inflating: reddit-dataset-master/README.md  
  inflating: reddit-dataset-master/entertainment_anime.csv  
  inflating: reddit-dataset-master/entertainment_comicbooks.csv  
  inflating: reddit-dataset-master/entertainment_harrypotter.csv  
  inflating: reddit-dataset-master/entertainment_movies.csv  
  inflating: reddit-dataset-master/entertainment_music.csv  
  inflating: reddit-dataset-master/entertainment_starwars.csv  
  inflating: reddit-dataset-master/gaming_dota2.csv  
  inflating: reddit-dataset-master/gaming_gaming.csv  
  inflating: reddit-dataset-master/gaming_leagueoflegends.csv  
  inflating: reddit-dataset-master/gaming_minecraft.csv  
  inflating: reddit-dataset-master/gaming_pokemon.csv  
  inflating: reddit-dataset-master/gaming_skyrim.csv  
  inflating: reddit-dataset-master/gaming_starcraft.csv  
  inflating: reddit-data

In [3]:
!ls -R

.:
CurlWget1684  reddit-dataset-master  sample_data

./reddit-dataset-master:
entertainment_anime.csv		learning_youshouldknow.csv
entertainment_comicbooks.csv	lifestyle_drunk.csv
entertainment_harrypotter.csv	lifestyle_food.csv
entertainment_movies.csv	lifestyle_frugal.csv
entertainment_music.csv		lifestyle_guns.csv
entertainment_starwars.csv	lifestyle_lifehacks.csv
gaming_dota2.csv		lifestyle_motorcycles.csv
gaming_gaming.csv		lifestyle_progresspics.csv
gaming_leagueoflegends.csv	lifestyle_sex.csv
gaming_minecraft.csv		news_conservative.csv
gaming_pokemon.csv		news_conspiracy.csv
gaming_skyrim.csv		news_libertarian.csv
gaming_starcraft.csv		news_news.csv
gaming_tf2.csv			news_offbeat.csv
headers.txt			news_politics.csv
humor_adviceanimals.csv		news_truereddit.csv
humor_circlejerk.csv		news_worldnews.csv
humor_facepalm.csv		README.md
humor_funny.csv			television_breakingbad.csv
humor_imgoingtohellforthis.csv	television_community.csv
humor_jokes.csv			television_doctorwho.csv
learning_a

# Installing dependencies

In [4]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [5]:
import nltk
nltk.download('stopwords')

!python3 -m spacy download en

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



# Imports

In [0]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [0]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# Data loading & preprocessing

In [8]:
#loading as pandas dataframe

df = pd.read_csv('./reddit-dataset-master/news_politics.csv')
df = df.dropna()
print(len(df))
df[:20]

85982


,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10
0,0,disclaimer i think obama should nominate some...,d028c5d,politics,news,1.455651e+09,degausse,3.0,0.0,1,1941,0.0
2,2,either way the process will be dragged out unt...,d026od6,politics,news,1.455648e+09,cyberspyder,5.0,0.0,1300,6462,1.0
3,3,republicans have always battled with severe ca...,d026wk3,politics,news,1.455649e+09,jabb0,15.0,0.0,44981,141501,0.0
4,4,politics were so different back then people on...,d02a3k3,politics,news,1.455653e+09,Hypertension123456,2.0,0.0,46,42036,0.0
5,5,equally ridiculous here s a 1970 law review a...,d025wad,politics,news,1.455647e+09,trash_reason,26.0,0.0,5613,1361,0.0
6,6,somehow i thought this would be about them wor...,d02buf9,politics,news,1.455655e+09,empress-of-blandings,1.0,0.0,843,55289,0.0
7,7,small government,d02d4bp,politics,news,1.455657e+09,brianwantsblood,1.0,0.0,1525,6122,0.0
8,8,let s clarify this headline during bush 13 ...,d02ful5,politics,news,1.455661e+09,GhostalMedia,1.0,0.0,47024,171635,1.0
9,9,yea the gop are such hypocrites,d02u1ga,politics,news,1.455683e+09,reboot108,1.0,0.0,1,247,0.0
11,11,i ve never met anyone in real life that knows ...,d02q5df,politics,news,1.455676e+09,Frederic_Bastiat,1.0,0.0,437,19960,1.0


In [9]:
#text preprocessing

# Convert to list
data = df['0'].values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['disclaimer i think obama should nominate someone and the senate should hold '
 'confirmation hearings but this article does not accurately describe most '
 'republicans statements on this issue the vast majority of republican '
 'statements although notably not the cruz one cited in the article carefully '
 'phrase the issue as whether a nominee has been nominated and confirmed in '
 'the president s last year in office although kennedy was confirmed in 1988 '
 'he was nominated by reagan at the end of 1987 so he does nt fall under this '
 'definition ']


In [10]:
#tokenizing text

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['disclaimer', 'think', 'obama', 'should', 'nominate', 'someone', 'and', 'the', 'senate', 'should', 'hold', 'confirmation', 'hearings', 'but', 'this', 'article', 'does', 'not', 'accurately', 'describe', 'most', 'republicans', 'statements', 'on', 'this', 'issue', 'the', 'vast', 'majority', 'of', 'republican', 'statements', 'although', 'notably', 'not', 'the', 'cruz', 'one', 'cited', 'in', 'the', 'article', 'carefully', 'phrase', 'the', 'issue', 'as', 'whether', 'nominee', 'has', 'been', 'nominated', 'and', 'confirmed', 'in', 'the', 'president', 'last', 'year', 'in', 'office', 'although', 'kennedy', 'was', 'confirmed', 'in', 'he', 'was', 'nominated', 'by', 'reagan', 'at', 'the', 'end', 'of', 'so', 'he', 'does', 'nt', 'fall', 'under', 'this', 'definition']]


In [11]:
#building bigram & trigram models for frequently occuring n-grams

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['disclaimer', 'think', 'obama', 'should', 'nominate', 'someone', 'and', 'the', 'senate', 'should', 'hold', 'confirmation_hearings', 'but', 'this', 'article', 'does', 'not', 'accurately_describe', 'most', 'republicans', 'statements', 'on', 'this', 'issue', 'the', 'vast', 'majority', 'of', 'republican', 'statements', 'although', 'notably', 'not', 'the', 'cruz', 'one', 'cited', 'in', 'the', 'article', 'carefully_phrase', 'the', 'issue', 'as', 'whether', 'nominee', 'has', 'been', 'nominated', 'and', 'confirmed', 'in', 'the', 'president', 'last', 'year', 'in', 'office', 'although', 'kennedy', 'was', 'confirmed', 'in', 'he', 'was', 'nominated', 'by', 'reagan', 'at', 'the', 'end', 'of', 'so', 'he', 'does', 'nt', 'fall', 'under', 'this', 'definition']


In [0]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['disclaimer', 'think', 'obama', 'nominate', 'someone', 'senate', 'hold', 'confirmation_hearing', 'article', 'accurately_describe', 'republican', 'statement', 'issue', 'vast', 'majority', 'republican', 'statement', 'notably', 'cruz', 'cite', 'article', 'carefully_phrase', 'issue', 'nominee', 'nominate', 'confirm', 'president', 'last', 'year', 'office', 'kennedy', 'confirm', 'nominate', 'reagan', 'end', 'nt', 'fall', 'definition']]


In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 3), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 1), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1)]]


In [15]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('accurately_describe', 1),
  ('article', 2),
  ('carefully_phrase', 1),
  ('cite', 1),
  ('confirm', 2),
  ('confirmation_hearing', 1),
  ('cruz', 1),
  ('definition', 1),
  ('disclaimer', 1),
  ('end', 1),
  ('fall', 1),
  ('hold', 1),
  ('issue', 2),
  ('kennedy', 1),
  ('last', 1),
  ('majority', 1),
  ('nominate', 3),
  ('nominee', 1),
  ('notably', 1),
  ('nt', 1),
  ('obama', 1),
  ('office', 1),
  ('president', 1),
  ('reagan', 1),
  ('republican', 2),
  ('senate', 1),
  ('someone', 1),
  ('statement', 2),
  ('think', 1),
  ('vast', 1),
  ('year', 1)]]

# LDA Model

In [0]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [17]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.077*"win" + 0.048*"hear" + 0.034*"show" + 0.033*"go" + 0.031*"wo" + '
  '0.022*"interesting" + 0.018*"insurance" + 0.016*"lie" + 0.016*"reality" + '
  '0.015*"rpolitic"'),
 (1,
  '0.070*"state" + 0.046*"man" + 0.039*"would" + 0.039*"call" + 0.034*"great" '
  '+ 0.029*"american" + 0.022*"city" + 0.022*"even" + 0.020*"use" + '
  '0.020*"stop"'),
 (2,
  '0.103*"want" + 0.040*"ca" + 0.039*"job" + 0.036*"shit" + 0.030*"something" '
  '+ 0.029*"anyone" + 0.025*"post" + 0.023*"nt" + 0.022*"seem" + '
  '0.020*"family"'),
 (3,
  '0.072*"big" + 0.054*"guy" + 0.047*"tell" + 0.042*"back" + 0.041*"take" + '
  '0.037*"poll" + 0.034*"probably" + 0.025*"video" + 0.024*"side" + '
  '0.019*"nevada"'),
 (4,
  '0.064*"hillary" + 0.056*"work" + 0.048*"many" + 0.042*"anything" + '
  '0.039*"use" + 0.025*"term" + 0.021*"news" + 0.020*"literally" + 0.019*"way" '
  '+ 0.018*"funny"'),
 (5,
  '0.096*"obama" + 0.074*"president" + 0.047*"woman" + 0.022*"thank" + '
  '0.019*"nomination" + 0.018*"hold" + 

In [18]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.931229046363098

Coherence Score:  0.35032270160538975


In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.127581  0.139378       1        1  13.388610
16     0.119034  0.117987       2        1   7.378056
18    -0.065626 -0.266112       3        1   6.205273
7      0.012371  0.033251       4        1   5.451911
2     -0.223698  0.115674       5        1   5.199551
11     0.053511 -0.090014       6        1   5.140554
13    -0.055617  0.001575       7        1   4.874213
5     -0.090143  0.108539       8        1   4.767360
19     0.062600 -0.069427       9        1   4.606367
8     -0.081343 -0.032751      10        1   4.541985
15     0.113610  0.081050      11        1   4.414512
1     -0.035827 -0.176571      12        1   4.305320
4     -0.203091  0.025442      13        1   4.251884
12     0.024706 -0.177842      14        1   3.997791
17     0.165046  0.145279      15        1   3.866604
9      0.158012  0.016796      16        1   3.812979
3      0.047561 -0.183985      17        1   3.654836
6     -0.179227  0.081181      18        1   3.562754
0      0.141324  0.115159      19        1   3.404582
10     0.164377  0.015389      20        1   3.174854, topic_info=     Category          Freq         Term         Total  loglift  logprob
19    Default  25774.000000           nt  25774.000000  30.0000  30.0000
267   Default  11619.000000       bernie  11619.000000  29.0000  29.0000
55    Default  13396.000000       people  13396.000000  28.0000  28.0000
517   Default  11674.000000       sander  11674.000000  27.0000  27.0000
47    Default  15087.000000        would  15087.000000  26.0000  26.0000
138   Default   7078.000000         want   7078.000000  25.0000  25.0000
252   Default   6879.000000        trump   6879.000000  24.0000  24.0000
98    Default  11947.000000          get  11947.000000  23.0000  23.0000
43    Default  11401.000000          say  11401.000000  22.0000  22.0000
28    Default   9457.000000        think   9457.000000  21.0000  21.0000
20    Default   6045.000000        obama   6045.000000  20.0000  20.0000
45    Default   5845.000000         vote   5845.000000  19.0000  19.0000
758   Default   6512.000000      clinton   6512.000000  18.0000  18.0000
424   Default   5277.000000       really   5277.000000  17.0000  17.0000
508   Default   5050.000000        party   5050.000000  16.0000  16.0000
168   Default  10010.000000           go  10010.000000  15.0000  15.0000
22    Default   7038.000000    president   7038.000000  14.0000  14.0000
161   Default   6619.000000         know   6619.000000  13.0000  13.0000
211   Default   4568.000000     campaign   4568.000000  12.0000  12.0000
589   Default   3984.000000        state   3984.000000  11.0000  11.0000
100   Default   5936.000000      hillary   5936.000000  10.0000  10.0000
181   Default   5688.000000         good   5688.000000   9.0000   9.0000
204   Default   8557.000000         make   8557.000000   8.0000   8.0000
170   Default   3494.000000          win   3494.000000   7.0000   7.0000
322   Default   3635.000000        right   3635.000000   6.0000   6.0000
210   Default   3479.000000          big   3479.000000   5.0000   5.0000
195   Default   4599.000000      support   4599.000000   4.0000   4.0000
530   Default   4240.000000        black   4240.000000   3.0000   3.0000
79    Default   4070.000000         bill   4070.000000   2.0000   2.0000
222   Default   3115.000000          pay   3115.000000   1.0000   1.0000
...       ...           ...          ...           ...      ...      ...
1641  Topic20    444.016235          amp    444.955902   3.4478  -4.5518
1054  Topic20    416.416321      address    417.355988   3.4477  -4.6160
1393  Topic20    362.502075  environment    363.441742   3.4473  -4.7546
1749  Topic20    326.192596         game    327.132263   3.4470  -4.8602
1111  Topic20    309.469513       ground    310.409180   3.4469  -4.9128
1548  Topic20    300.736053   interested    301.675720   3.4468  -4.